# Simulating the Effects of TN Business Tax Reform on Investment Incentives
### Jason DeBacker and Richard W. Evans, December 22, 2022

Important features of TN business tax system:
* 3 components:
  1. "Excise Tax"
    * Essentailly a CIT
    * Rate is 6.5%
    * Depreciation handled under same rules as IRC *EXCEPT* no bonus depreciation.
  2. Franchise tax
    * Tax on the value of assets of the business
    * Rate is 0.26%
  3. Business tax
    * Tax on the gross receipts of the business
    * Some industries (e.g., utilities) handled differently
    * Rate varies (see [here, p. 62](https://www.tn.gov/content/dam/tn/revenue/documents/tax_manuals/august-2022/Business-Tax.pdf))
      * Varies between 0.02% and 0.3% (depends on industry)


  Computing the cost-of-capital at the state-level:
  * the statutory rate used will be the sum of the federal and state CIT
    * But can deduct state taxes at federal level
    * So maybe rate is Fed Rate + State rate - fed rate * state rate?
  * franchise tax can use the wealth tax parameter in CCC
  * The interaction of state and federal is not something I want to rush into CCC, so we'll do most of the calculations here by functions defined in this notebook (with a bit of assistance from a couple CCC functions.


Equation for the cost of capital at when considering state taxes in TN:
$$
\rho = \left[\frac{(r-\pi-\delta)(1-k)(1-u^fz^f-u^sz^s + u^fu^sz^s)}{(1-u^f-u^s+u^fu^s)} + w \right] / (1-\tau^{GR})
$$
where:
* $\rho$ = the cost of capital
* $r$ =the nominal discount rate
* $\pi$ = the inflation rate
* $k$ = the investment tax credit rate
* $u^f$ = the statutory CIT rate at the federal level 
* $u^s$ = the statutory CIT rate at the state level 
* $z^f$ = the NPV of depreciation deduction under federal system
* $z^s$ = the NPV of depreciation deduction under state system
* $w$ = the property tax rate
* $\tau^{GR}$ = the gross receipts tax rate

Calibration:

| Parameter     | Value | Source |
| ----------- | ----------- |----------- |
| $r$     | 0.06      | To give real return of 4\%|
| $\pi$  | 0.02      | Fed target|
| $k$  | 0.0        | No general invest tax credit at federal or state level|
| $u^f$  | 0.21       | Federal IRC|
| $u^s$  | 0.065       | Excise tax rate in TN law|
| $z^f$  | Varies        | Federal IRC|
| $z^s$  |   Varies      | Like Federal IRC, but no bonus deprec|
| $w$  | 0.0026       | Franchise tax rate in TN law |
| $\tau^{GR}$  | 0.0002-0.003       | Bus tax rate in TN law|



In [ ]:
!pip install cost-of-capital-calculator &> /dev/null
!pip install taxcalc &> /dev/null 
!pip install bokeh &> /dev/null

Now we'll import the Python packages and modules we'll work with:

In [ ]:
# imports
import ccc
import taxcalc as tc
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
# set template for plotting
pio.templates.default = "plotly_white"

## Setup

Create objects to work with

In [ ]:
# Creating an instance of the Assets class and call it "assets"
assets = ccc.data.Assets()
# Create an instance of the DepreciationParams
# class object and call it dp
dp = ccc.parameters.DepreciationParams()

In [ ]:
# define new cost of capital function to take into account state taxes
def eq_coc_state(delta, z_f, z_s, w, u_f, u_s, tau_GR, inv_tax_credit, pi, r):
    r'''
    Compute the cost of capital
    .. math::
        \rho = \frac{\frac{(r-\pi+\delta)}{1-u^f-u^s+u^fu^s}(1-u^fz^f-u^sz^s+u^fu^sz^s)+w}{(1-\tau^GR)}-\delta
    Args:
        delta (array_like): rate of economic depreciation
        z_f (array_like): net present value of depreciation deductions for
            $1 of investment under the federal system
        z_s (array_like): net present value of depreciation deductions for
            $1 of investment under the state system
        w (scalar): property tax rate
        u_f (scalar): statutory marginal tax rate for the first layer of
            income taxes at the federal level
        u_s (scalar): statutory marginal tax rate for the first layer of
            income taxes at the state level
        tau_GR (scalar): gross receipts tax
        inv_tax_credit (scalar): investment tax credit rate
        pi (scalar): inflation rate
        r (scalar): discount rate
    Returns:
        rho (array_like): the cost of capital
    '''
    rho = (((((r - pi + delta) / (1 - u_f - u_s + u_f * u_s)) *
           (1 - inv_tax_credit - u_f * z_f - u_s * z_s) + w) / (1 - tau_GR)) - delta)

    return rho

In [ ]:
# Define model parameters
inflation_rate = 0.02 # Inflation rate
nominal_int_rate = 0.06
fraction_financed_w_debt = 0.0
federal_bonus_depreciation = {"machines": 1.0, "buildings": 0.0, "intangibles": 1.0}
depreciation_rates = {"machines": 0.1031, "buildings": 0.0314, "intangibles": 0.33}
# Example machine: EI40, example building: SI00, example intangible: ENS3
depreciation_lives = {"machines": 7, "buildings": 39, "intangibles": 3}
depreciation_methods = {"machines": "dbsl", "buildings": "sl", "intangibles": "sl"}
E = 0.06  # Expected after-tax return on corporate equity
profit_rate = 0.2 # 0.2 is a 20% profit rate
int_haircut = 0.0
u_f = 0.21
u_s = 0.065
franchise_tax_rate = 0.0026
tau_GR = 0.003 # this is the higher end 0.0002-0.003 is what TF reports, varies by industry
inv_tax_credit = 0.0
bonus_s = {"machines": 0.0, "buildings": 0.0, "intangibles": 0.0}


In [ ]:
# compute outputs
def compute_outputs(u_s, bonus_s, franchise_tax_rate, tau_GR):
    """
    This function computes the outputs of interest and allows one to change the
    parameters of TN state law

    Args:
      u_s (scalar): statutory marginal tax rate for the first layer of
            income taxes at the state level
      bonus_s (dict): rates of bonus depreciation in TN by asset type
      franchise_tax_rate (scalar): TN franchise tax
      tau_GR (scalar): gross receipts tax
    """
    out_dict = {"machines": {}, "buildings": {}, "intangibles": {}}
    for k, v in depreciation_rates.items():
        r = ccc.paramfunctions.calc_r(
            u_f + u_s - u_f * u_s, nominal_int_rate, inflation_rate,
            nominal_int_rate, fraction_financed_w_debt, int_haircut, E, 0.0)
        r_prime = ccc.paramfunctions.calc_r_prime(
            nominal_int_rate, inflation_rate, fraction_financed_w_debt, E)
        if depreciation_methods[k] == "dbsl":
            z_f = ccc.calcfunctions.dbsl(depreciation_lives[k], 2, federal_bonus_depreciation[k], r)
            z_s = ccc.calcfunctions.dbsl(depreciation_lives[k], 2, bonus_s[k], r) 
        elif depreciation_methods[k] == "sl":
            z_f = ccc.calcfunctions.sl(depreciation_lives[k], federal_bonus_depreciation[k], r)
            z_s = ccc.calcfunctions.sl(depreciation_lives[k], bonus_s[k], r) 
        else:
          print("Please enter one of: dbsl, sl")
          assert False
        rho = eq_coc_state(
                depreciation_rates[k], z_f, z_s, franchise_tax_rate,
                u_f, u_s, tau_GR, inv_tax_credit, inflation_rate, r)
        metr = ccc.calcfunctions.eq_metr(rho, r_prime, inflation_rate)
        eatr = ccc.calcfunctions.eq_eatr(rho, metr, profit_rate, u_f + u_f - u_f * u_s)
        out_dict[k]["rho"] = rho
        out_dict[k]["metr"] = metr
        out_dict[k]["eatr"] = eatr
    return out_dict

In [ ]:
base_df = pd.DataFrame(compute_outputs(u_s, bonus_s, franchise_tax_rate, tau_GR))
repealGR_df = pd.DataFrame(compute_outputs(u_s, bonus_s, franchise_tax_rate, 0.0))
fedbonus_df = pd.DataFrame(compute_outputs(u_s, federal_bonus_depreciation, franchise_tax_rate, tau_GR))
repealFT_df = pd.DataFrame(compute_outputs(u_s, bonus_s, 0.0, tau_GR))

In [ ]:
# Create policy names and put in one dataframe
base_df['Policy'] = "Current Law"
repealGR_df['Policy'] = "Repeal Business Tax"
fedbonus_df['Policy'] = "Follow Federal Bonus Depreciation"
repealFT_df['Policy'] = "Repeal Franchise Tax"
# append dataframes together
df = pd.concat([base_df, repealGR_df, fedbonus_df, repealFT_df])
df.reset_index(inplace=True)
df.rename(columns={"index": "output_var"}, inplace=True)
df = pd.melt(df, id_vars=["Policy", "output_var"], var_name="asset_type")

In [ ]:
# Plot results
fig = px.bar(df[(df["output_var"]=="metr") & (df["asset_type"]=="machines")], x="Policy", y="value",
             color='Policy', labels={'value':'Marginal Effective Tax Rate'}, height=400)
fig.show()

In [ ]:
# Plot results
fig = px.bar(df[(df["output_var"]=="rho") & (df["asset_type"]=="machines")], x="Policy", y="value",
             color='Policy', labels={'value':'Cost of Capital'}, height=400)
fig.show()

In [ ]:
fig = px.bar(df[(df["output_var"]=="metr") & (df["asset_type"]=="machines")], x="value", y="Policy",
             color='Policy', labels={'value':'Marginal Effective Tax Rate'}, height=400)
fig.show()

In [ ]:
fig = px.bar(df[(df["output_var"]=="metr") & (df["asset_type"]=="machines")], x="Policy", y="value",
             color='Policy', labels={'value':'Marginal Effective Tax Rate'}, height=400)
fig.show()

In [ ]:
# Plot results
fig = px.histogram(df[(df["output_var"]=="metr") & (df['Policy'].isin(["Current Law", "Repeal Business Tax"]))], x="asset_type", y="value",
             color='Policy', barmode='group', labels={'asset_type':'Asset Type', 'value': 'Marginal Effective Tax Rate'},
             height=400)
fig.show()

In [ ]:
# compute example results for equipment
r = ccc.paramfunctions.calc_r(
        u_f + u_s - u_f * u_s, nominal_int_rate, inflation_rate,
        nominal_int_rate, fraction_financed_w_debt, int_haircut, E, 0.0)
r_prime = ccc.paramfunctions.calc_r_prime(
    nominal_int_rate, inflation_rate, fraction_financed_w_debt, E)
z_f = ccc.calcfunctions.dbsl(5, 2, 1.0, r)  # 5 year asset, double declining balance, 100% bonus
z_s = ccc.calcfunctions.dbsl(5, 2, 0.0, r)  # 5 year asset, double declining balance, no bonus
rho = eq_coc_state(
        depreciation_rates['machines'], z_f, z_s, franchise_tax_rate,
        u_f, u_s, tau_GR, inv_tax_credit, inflation_rate, r)
metr = ccc.calcfunctions.eq_metr(rho, r_prime, inflation_rate)
eatr = ccc.calcfunctions.eq_eatr(rho, metr, profit_rate, u_f + u_f - u_f * u_s)
print('Machines: rho = ', rho, ', metr = ', metr, ' and eatr = ', eatr)

In [ ]:
# compare to federal
r = ccc.paramfunctions.calc_r(
        u_f + u_s - u_f * u_s, nominal_int_rate, inflation_rate,
        nominal_int_rate, fraction_financed_w_debt, int_haircut, E, 0.0)
r_prime = ccc.paramfunctions.calc_r_prime(
    nominal_int_rate, inflation_rate, fraction_financed_w_debt, E)
z_f = ccc.calcfunctions.dbsl(5, 2, 1.0, r)  # 5 year asset, double declining balance, 100% bonus
rho = ccc.calcfunctions.eq_coc(
        depreciation_rates['machines'], z_f, 0.0,
        u_f, inv_tax_credit, inflation_rate, r)
metr = ccc.calcfunctions.eq_metr(rho, r_prime, inflation_rate)
eatr = ccc.calcfunctions.eq_eatr(rho, metr, profit_rate, u_f + u_f - u_f * u_s)
print('Machines: rho = ', rho, ', metr = ', metr, ' and eatr = ', eatr)

In [ ]:
# Create output DataFrame
out_df = df[[
    'country', 'corporate_rate', 'property_tax', 'r_and_d_credit',
    'allowance_corporate_equity', 'machines_cost_recovery',
    'buildings_cost_recovery', 'intangibles_cost_recovery']].copy()
# Compute intermediate inputs
out_df['r'] = ccc.paramfunctions.calc_r(
        out_df['corporate_rate'], nominal_int_rate, inflation_rate,
        ace_int_rate, fraction_financed_w_debt, int_haircut, E, out_df['allowance_corporate_equity'])
out_df['r_prime'] = ccc.paramfunctions.calc_r_prime(
    nominal_int_rate, inflation_rate, fraction_financed_w_debt, E)
# Compute final outputs
for k, v in depreciation_rates.items():
    if k == 'intangibles':
        inv_tax_credit = out_df['r_and_d_credit']  # apply R&D credit to intangibles
    else:
        inv_tax_credit = 0.0
    out_df['coc_' + k] = ccc.calcfunctions.eq_coc(
        v, out_df[k + '_cost_recovery'], prop_tax_rate,
        out_df['corporate_rate'], inv_tax_credit, inflation_rate, out_df['r'])
    out_df['metr_' + k] = ccc.calcfunctions.eq_metr(
        out_df['coc_' + k], out_df['r_prime'], inflation_rate)
    out_df['eatr_' + k] = ccc.calcfunctions.eq_eatr(
         out_df['coc_' + k],  out_df['metr_' + k], profit_rate, out_df['corporate_rate'])

In [ ]:
# plot METRs by country
out_df.sort_values(by='metr_machines', inplace=True)
fig = px.bar(out_df, x='country', y='metr_machines')
fig.show()

In [ ]:
df

## PEW State Balances Data

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from bokeh.io import output_file, output_notebook, export_png
from bokeh.plotting import figure, show
from bokeh.models import (ColumnDataSource, Title, Legend, HoverTool,
                          NumeralTickFormatter)
from bokeh.models.tickers import SingleIntervalTicker
from bokeh.models.annotations import Label

### PEW state balances data: Rainy day fund balances

In [2]:
# Read in the PEW data on rainy day funds by year and
# state from worksheet
rain_df = pd.read_excel(
    "data/ReservesBalancesData.xlsx",
    sheet_name="Rainy Day Fund Data",
    header=5,
    index_col=0,
    skipfooter=17
)
rain_df.replace(0, np.nan, inplace=True)

# Create a DataFrame of just the rainy day funds in $millions
# by state and by year
rain_dol_df = rain_df.loc[:'Wyoming', 'FY 2000.1':'FY 2022 (estimated).1']
rain_dol_df.rename(columns = {
    'FY 2000.1': '2000',
    'FY 2001.1': '2001',
    'FY 2002.1': '2002',
    'FY 2003.1': '2003',
    'FY 2004.1': '2004',
    'FY 2005.1': '2005',
    'FY 2006.1': '2006',
    'FY 2007.1': '2007',
    'FY 2008.1': '2008',
    'FY 2009.1': '2009',
    'FY 2010.1': '2010',
    'FY 2011.1': '2011',
    'FY 2012.1': '2012',
    'FY 2013.1': '2013',
    'FY 2014.1': '2014',
    'FY 2015.1': '2015',
    ' FY 2016.1': '2016',
    ' FY 2017.1': '2017',
    ' FY 2018.1': '2018',
    'FY 2019.1': '2019',
    'FY 2020.1': '2020',
    'FY 2021.1': '2021',
    'FY 2022 (estimated).1': '2022',
}, inplace = True)

# Create a DataFrame of just the rainy day funds as a percent of
# general fund expenditures by state and by year
rain_pct_df = rain_df.loc[:, 'FY 2000.2':'FY 2022 (estimated).2']
rain_pct_df.rename(columns = {
    'FY 2000.2': '2000',
    'FY 2001.2': '2001',
    'FY 2002.2': '2002',
    'FY 2003.2': '2003',
    'FY 2004.2': '2004',
    'FY 2005.2': '2005',
    'FY 2006.2': '2006',
    'FY 2007.2': '2007',
    'FY 2008.2': '2008',
    'FY 2009.2': '2009',
    'FY 2010.2': '2010',
    'FY 2011.2': '2011',
    'FY 2012.2': '2012',
    'FY 2013.2': '2013',
    'FY 2014.2': '2014',
    'FY 2015.2': '2015',
    ' FY 2016.2': '2016',
    ' FY 2017.2': '2017',
    ' FY 2018.2': '2018',
    'FY 2019.2': '2019',
    'FY 2020.2': '2020',
    'FY 2021.2': '2021',
    'FY 2022 (estimated).2': '2022',
}, inplace = True)

/var/folders/d4/trj3dssd6s3g8kxvjmczz11w0000gn/T/ipykernel_9171/2235183987.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rain_pct_df.rename(columns = {


### PEW state balances data: Total balances

In [3]:
# Read in the PEW data on total reserves and balances
# by year and state from worksheet
totbal_df = pd.read_excel(
    "data/ReservesBalancesData.xlsx",
    sheet_name="Total Balances Data",
    header=5,
    index_col=0,
    skipfooter=17
)
totbal_df.replace(0, np.nan, inplace=True)

# Create a DataFrame of just the total reserves and balances
# funds in $millions by state and by year
totbal_dol_df = totbal_df.loc[:'Wyoming', 'FY 2000.1':'FY 2022 (estimated).1']
totbal_dol_df.rename(columns = {
    'FY 2000.1': '2000',
    'FY 2001.1': '2001',
    'FY 2002.1': '2002',
    'FY 2003.1': '2003',
    'FY 2004.1': '2004',
    'FY 2005.1': '2005',
    'FY 2006.1': '2006',
    'FY 2007.1': '2007',
    'FY 2008.1': '2008',
    'FY 2009.1': '2009',
    'FY 2010.1': '2010',
    'FY 2011.1': '2011',
    'FY 2012.1': '2012',
    'FY 2013.1': '2013',
    'FY 2014.1': '2014',
    'FY 2015.1': '2015',
    ' FY 2016.1': '2016',
    ' FY 2017.1': '2017',
    ' FY 2018.1': '2018',
    'FY 2019.1': '2019',
    'FY 2020.1': '2020',
    'FY 2021.1': '2021',
    'FY 2022 (estimated).1': '2022',
}, inplace = True)

# Create a DataFrame of just the total reserves and balances funds
# as a percent of general fund expenditures by state and by year
totbal_pct_df = totbal_df.loc[:, 'FY 2000.2':'FY 2022 (estimated).2']
totbal_pct_df.rename(columns = {
    'FY 2000.2': '2000',
    'FY 2001.2': '2001',
    'FY 2002.2': '2002',
    'FY 2003.2': '2003',
    'FY 2004.2': '2004',
    'FY 2005.2': '2005',
    'FY 2006.2': '2006',
    'FY 2007.2': '2007',
    'FY 2008.2': '2008',
    'FY 2009.2': '2009',
    'FY 2010.2': '2010',
    'FY 2011.2': '2011',
    'FY 2012.2': '2012',
    'FY 2013.2': '2013',
    'FY 2014.2': '2014',
    'FY 2015.2': '2015',
    ' FY 2016.2': '2016',
    ' FY 2017.2': '2017',
    ' FY 2018.2': '2018',
    'FY 2019.2': '2019',
    'FY 2020.2': '2020',
    'FY 2021.2': '2021',
    'FY 2022 (estimated).2': '2022',
}, inplace = True)

/var/folders/d4/trj3dssd6s3g8kxvjmczz11w0000gn/T/ipykernel_9171/3735465516.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totbal_pct_df.rename(columns = {


### Figure 1. Plot time series of rainy day funds and total balances as percent of general fund expenditures for both the 50-state median and for Tennessee
Executing the cell below will create the `rain_totbal_50_tn_timeseries.html` file in your `/images/` folder and open that file as a browser window. I created the `.png` version of the file by just screenshotting the `.html` image.

In [17]:
rain_pct_50_df = rain_pct_df.loc["50-state median", :].to_frame().reset_index()
rain_pct_50_df.rename(columns = {"index":"year", "50-state median":"fraction"}, inplace = True)
rain_pct_50_df["percent"] = 100 * rain_pct_50_df["fraction"]

rain_pct_tn_df = rain_pct_df.loc["Tennessee", :].to_frame().reset_index()
rain_pct_tn_df.rename(columns = {"index":"year", "Tennessee":"fraction"}, inplace = True)
rain_pct_tn_df["percent"] = 100 * rain_pct_tn_df["fraction"]

totbal_pct_50_df = totbal_pct_df.loc["50-state median", :].to_frame().reset_index()
totbal_pct_50_df.rename(columns = {"index":"year", "50-state median":"fraction"}, inplace = True)
totbal_pct_50_df["percent"] = 100 * totbal_pct_50_df["fraction"]

totbal_pct_tn_df = totbal_pct_df.loc["Tennessee", :].to_frame().reset_index()
totbal_pct_tn_df.rename(columns = {'index':'year', "Tennessee":"fraction"}, inplace = True)
totbal_pct_tn_df["percent"] = 100 * totbal_pct_tn_df["fraction"]

fig_title = ("Rainy Day fund and total reserves as a percentage of " +
             "general fund expenditures: 2000-2022")
output_file("./images/rain_totbal_50_tn_timeseries.html", title=fig_title)
min_year = 2000
max_year = 2022
min_pct = 2.0
max_pct = 0.0
pct_buffer = 0.05

var_list = [rain_pct_50_df, rain_pct_tn_df, totbal_pct_50_df, totbal_pct_tn_df]
color_list = ["blue", "orange", "blue", "orange"]
marker_list = ["circle", "circle", "square", "square"]
legend_label_list = ["Rainy Day Fund, 50-state median", "Rainy Day Fund, Tennessee",
                     "Total balances, 50-state median", "Total balances, Tennessee"]
cds_list = []

for k, df in enumerate(var_list):
    min_pct = np.minimum(min_pct, df["percent"].min())
    max_pct = np.maximum(max_pct, df["percent"].max())
    cds_list.append(ColumnDataSource(df))

fig = figure(title=fig_title,
             height=600,
             width=1000,
             x_axis_label='Year',
             x_range=(min_year - 1, max_year + 1),
             y_axis_label='Percent of general fund expenditures',
             y_range=(min_pct - pct_buffer * (max_pct - min_pct),
                      max_pct + pct_buffer * (max_pct - min_pct)),
             toolbar_location=None)

# Set title font size and axes font sizes
fig.title.text_font_size = '15pt'
fig.xaxis.axis_label_text_font_size = '12pt'
fig.xaxis.major_label_text_font_size = '12pt'
fig.yaxis.axis_label_text_font_size = '12pt'
fig.yaxis.major_label_text_font_size = '12pt'

# Modify tick intervals for X-axis and Y-axis
fig.xaxis.ticker = SingleIntervalTicker(interval=2, num_minor_ticks=2)
fig.xgrid.ticker = SingleIntervalTicker(interval=2)
fig.yaxis.ticker = SingleIntervalTicker(interval=10, num_minor_ticks=5)
fig.ygrid.ticker = SingleIntervalTicker(interval=10)

# Create lines and markers for time series
for k, yvar in enumerate(var_list):
    fig.line(x='year', y='percent', source=cds_list[k], color=color_list[k],
            line_width=3, alpha=0.7)
    fig.scatter(x='year', y='percent', source=cds_list[k], size=8,
                line_width=1, line_color='black', fill_color=color_list[k],
                marker=marker_list[k], line_alpha=0.7, fill_alpha=0.7,
                legend_label=legend_label_list[k])

fig.segment(x0=2021.2, y0=min_pct - pct_buffer * (max_pct - min_pct),
            x1=2021.2, y1=max_pct + pct_buffer * (max_pct - min_pct),
            color='gray', line_dash='6 2', line_width=2)
    
label_temp = Label(x=2021.3, y=30.0, x_units='data', y_units='data',
                   text='Projected', text_font_size='4mm')
fig.add_layout(label_temp)

# Add information on hover
tooltips = [('Year', '@year'),
            ('Pct of general fund exps.','@percent{0.0}' + '%')]
fig.add_tools(HoverTool(tooltips=tooltips, toggleable=False))

# Add legend
fig.legend.location = 'top_center'
fig.legend.border_line_width = 1
fig.legend.border_line_color = 'black'
fig.legend.border_line_alpha = 1
fig.legend.label_text_font_size = '4mm'

# Add notes below image
note_text_list = [
    (
        'Source: PEW Charitable Trusts, "Fiscal 50: State Trends and Analysis: ' +
        'Reserves and Balances", updated Dec. 16, 2022'
    ),
    ('        (accessed Dec. 31, 2022).')    
]
for note_text in note_text_list:
    caption = Title(text=note_text, align='left', text_font_size='4mm',
                    text_font_style='italic')
    fig.add_layout(caption, 'below')

# # This export_png() function requires selenium package as well as firefox
# # and geckodriver packages
# # (see https://docs.bokeh.org/en/3.0.3/docs/user_guide/output/export.html)
# export_png(fig, filename="/images/rain_totbal_50_tn_timeseries.png")

show(fig)

#### Bokeh

#### Matplotlib

In [ ]:
plt.plot(
    np.arange(2000, 2023), rain_pct_50, '--', color="blue",
    label="Rainy day fund: 50-state median"
)
plt.plot(
    np.arange(2000, 2023), rain_pct_tn, '--', color="orange",
    label="Rainy day fund: Tennessee"
)
plt.plot(
    np.arange(2000, 2023), totbal_pct_50, ':', color="blue",
    label="Total reserves and balances: 50-state median"
)
plt.plot(
    np.arange(2000, 2023), totbal_pct_tn, ':', color="orange",
    label="Total reserves and balances: Tennessee"
)
plt.legend(loc="upper left")
plt.title(r"Rainy Day fund and total reserves as a percentage of general fund expenditures: 2000-2022")
plt.xlabel(r"Year")
plt.ylabel(r"Percent")